# 第五次课后练习题

## 问题重述

In [119]:
from scipy.optimize import approx_fprime
import numpy as np

# 相关数据
t = np.array([4.0000, 2.0000, 1.0000, 0.5000, 0.2500, 0.1670, 0.1250,
              0.1000, 0.0833, 0.0714, 0.0625])
y = np.array([0.1957, 0.1947, 0.1735, 0.1600, 0.0844, 0.0627, 0.0456,
              0.0342, 0.0323, 0.0235, 0.0246])
# 目标函数
def f(x):
    x1, x2, x3, x4 = x
    return np.sum((x1 * (t**2 + x2*t)) / (t**2 + x3*t + x4) - y)**2

def f_1(x):
    x1, x2, x3, x4 = x
    return y - x1 * (t**2 + x2 * t) / (t**2 + x3 * t + x4)

# 梯度函数
def g(x):
    eps = 1e-8
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x1 = np.copy(x)
        x2 = np.copy(x)
        x1[i] += eps
        x2[i] -= eps
        grad[i] = (f(x1) - f(x2)) / (2 * eps)
    return grad
# 黑塞矩阵
def G(x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            x_ijp = np.array(x, dtype=float)
            x_ijp[i] += h
            x_ijp[j] += h
            x_ip = np.array(x, dtype=float)
            x_ip[i] += h
            x_jp = np.array(x, dtype=float)
            x_jp[j] += h
            x_orig = np.array(x, dtype=float)
            H[i, j] = (f(x_ijp) - f(x_ip) - f(x_jp) + f(x_orig)) / (h**2)
    return H
# 雅可比矩阵
def J(x, t = t):
    x1, x2, x3, x4 = x
    J = np.zeros((len(t), len(x)))
    for i, t in enumerate(t):
        d1 = (t**2 + x2 * t) / (t**2 + x3 * t + x4)
        d2 = x1 * t / (t**2 + x3 * t + x4)
        d3 = -x1 * (t**2 + x2 * t) * t / (t**2 + x3 * t + x4)**2
        d4 = -x1 * (t**2 + x2 * t) / (t**2 + x3 * t + x4)**2
        J[i, :] = [-d1, -d2, -d3, -d4]
    return J

x0 = [0.2, 0.2, 0.1, 0.1]

## 最速下降法

In [120]:
import numpy as np
import time

def Armijo(f, g, xk, dk, alpha = 1, sigma = 0.1, beta = 0.5):
    while f(xk + alpha * dk) > f(xk) + sigma * alpha * np.dot(dk, g(xk)):
        alpha *= beta
    return alpha

def SDmethod(f, g, x0, tol = 1e-6):
    start = time.time()
    xk = x0
    items = 0
    while np.linalg.norm(g(xk)) >= tol:
        dk = - g(xk)
        alpha = Armijo(f, g, xk, dk)
        xk += alpha * dk
        items += 1
    end = time.time()
    return xk, f(xk), items, end - start


x_min, f_min, items, time_cost = SDmethod(f, g, x0)
print(f"最优解:",x_min,'\n',"最小值:",f_min,'\n',"迭代次数:",items,'\n',"计算时间:",time_cost)

最优解: [0.18067445 0.19332725 0.10365823 0.11497046] 
 最小值: 4.567515238711441e-15 
 迭代次数: 14 
 计算时间: 0.02473306655883789


## 牛顿法

In [121]:
import time

def NewtonMethod(f, g, G, x0, tol=1e-6):
    start = time.time()
    xk = x0
    items = 0
    while np.linalg.norm(g(xk)) >= tol:
        try:
            # 尝试计算G(xk)的逆
            G_inv = np.linalg.inv(G(xk))
        except np.linalg.LinAlgError:
            print("G(xk) is singular, switching to steepest descent method.")
            return SDmethod(f, g, xk)  

        # 更新xk
        xk -= G_inv.dot(g(xk))
        items += 1

    end = time.time()
    return xk, f(xk), items, end - start

# 调用NewtonMethod函数
x_min, f_min, items, time_cost = NewtonMethod(f, g, G, x0)
print(f"最优解: {x_min}\n最小值: {f_min}\n迭代次数: {items}\n计算时间: {time_cost}")


最优解: [ 0.16861388  0.1264594  -0.00553715  0.08761418]
最小值: 4.102219949098878e-20
迭代次数: 3
计算时间: 0.005000114440917969


## 阻尼牛顿法

In [122]:
import time
import numpy as np

def DampedNewtonMethod(f, g, G, x0, tol=1e-5):
    start = time.time()
    xk = x0
    items = 0
    while np.linalg.norm(g(xk)) >= tol:
        Gk = G(xk)
        gk = g(xk)

        if np.all(np.linalg.eigvals(Gk) <= 0):
            print("Matrix G(xk) is not positive definite, switching to steepest descent method.")
            return SDmethod(f, g, xk) 

        try:
            dk = -np.linalg.solve(Gk, gk) 
        except np.linalg.LinAlgError:
            print("Matrix G(xk) is singular, switching to steepest descent method.")
            return SDmethod(f, g, xk)

        alpha = Armijo(f, g, xk, dk)  
        xk += alpha * dk
        items += 1

    end = time.time()
    return xk, f(xk), items, end - start

# 调用DampedNewtonMethod函数
x_min, f_min, items, time_cost = DampedNewtonMethod(f, g, G, x0)
print(f"最优解: {x_min}\n最小值: {f_min}\n迭代次数: {items}\n计算时间: {time_cost}")


最优解: [ 0.16861388  0.1264594  -0.00553714  0.08761418]
最小值: 4.1021936681749444e-20
迭代次数: 3
计算时间: 0.014790534973144531


## 拟牛顿法 
#### (这里采用BFGS方法)

In [123]:
import time

def BFGSmethod(f, g, x0, tol=1e-5):
    start = time.time()
    n = len(x0)
    xk = x0
    B = np.eye(n)  # 初始化黑塞逆矩阵为单位阵
    items = 0  # 初始化迭代次数计数器
    while np.linalg.norm(g(xk)) >= tol:
        grad = g(xk)
        dk = -np.dot(B, grad)
        alpha = Armijo(f, g, xk, dk)  
        xk += alpha * dk
        items += 1
        
        s = alpha * dk
        y = g(xk) - grad
        Bs = np.dot(B, s)
        Bs_dot_s = np.dot(Bs, s)
        if np.abs(Bs_dot_s) > 1e-10:  
            B += np.outer(y, y) / np.dot(y, s) - np.outer(Bs, Bs) / Bs_dot_s
    
    end = time.time()
    return xk, f(xk), items, end - start

x_min, f_min, items, time_cost = BFGSmethod(f, g, x0)
print(f"最优解: {x_min}\n最小值: {f_min}\n迭代次数: {items}\n计算时间: {time_cost}")


最优解: [0.17988071 0.19365835 0.10341633 0.11357361]
最小值: 4.003777718663273e-14
迭代次数: 7
计算时间: 0.013330221176147461


## 共轭梯度法


In [124]:
import time

def Conjugate_Gradient(f, g, x0, method='HS', max_iter=10000, tol=1e-6):
    start = time.time()
    xk = x0
    grad = g(xk)
    dk = -grad
    items = 0
    beta_func = {
        'HS': lambda g_next, g, dk: np.dot(g_next, g_next - g) / np.dot(dk, g_next - g),
        'PRP': lambda g_next, g: np.dot(g_next, g_next - g) / np.dot(g, g),
        'FR': lambda g_next, g: np.dot(g_next, g_next) / np.dot(g, g),
        'CD': lambda g_next, g, dk: -np.dot(g_next, g_next) / np.dot(dk, g),
        'DY': lambda g_next, g, dk: np.dot(g_next, g_next - g) / np.dot(dk, g_next - g),
        'PRP_plus': lambda g_next, g: max(0, np.dot(g_next, g_next - g) / np.dot(g, g))
    }[method]
    while np.linalg.norm(grad) >= tol and items < max_iter:
        alpha = Armijo(f, g, xk, dk)  # 确保Armijo函数已经正确实现
        x_next = xk + alpha * dk
        g_next = g(x_next)
        beta = beta_func(g_next, grad, dk)
        dk = -g_next + beta * dk
        xk, grad = x_next, g_next
        items += 1
    end = time.time()
    return xk, f(xk), items, end - start

# 调用共轭梯度法
x_min, f_min, items, time_cost = Conjugate_Gradient(f, g, x0)
print(f"最优解: {x_min}\n最小值: {f_min}\n迭代次数: {items}\n计算时间: {time_cost}")


最优解: [0.26449945 0.14643722 0.13821816 0.31970571]
最小值: 4.403497895091725e-16
迭代次数: 16
计算时间: 0.04056906700134277


## LMF方法

In [125]:
import time 

def LMFmethod(f, j, x0, max_iter=100, tol=1e-6, lambda_ = 0.01, nu = 2.0):
    start = time.time()
    xk = np.array(x0)
    for items in range(max_iter):
        r = f_1(xk)
        J = j(xk)
        
        A = J.T @ J
        g = J.T @ r
        I = np.eye(len(xk))
        
        delta = np.linalg.solve(A + lambda_ * I, -g)
        
        if np.linalg.norm(delta) < tol:
            break
        
        x_new = xk + delta
        if np.sum(f_1(x_new)**2) < np.sum(r**2):
            xk = x_new
            lambda_  /=nu
        else:
            lambda_ *= nu
    end = time.time()
    
    return xk, f(xk), items, end - start

# Using the LMF method
x_min, f_min, items, time_cost = LMFmethod(f, J, x0)
print(f"最优解: {x_min}\n最小值: {f_min}\n迭代次数: {items}\n计算时间: {time_cost}")

最优解: [0.19280692 0.19128283 0.12305666 0.13606256]
最小值: 7.297756577816817e-06
迭代次数: 7
计算时间: 0.0
